<a href="https://colab.research.google.com/github/gvelec/AppliedDataScience/blob/master/PGA_Segment%26Clust_Toronto_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto


> In the next cells I propose a code in python according with the instruccions for this peer-graded assignement.



First I need to load the libraries required to scrap the data.

In [126]:
# WEB SCRAPING
# Table from Wikipedia
#### Finding ZIP Code, Borough, and Neighbourhood
#
# loading the libraries required
from time import time
from IPython.core.display import clear_output
from warnings import warn
from bs4 import BeautifulSoup
import requests
import pandas as pd
#

In this point I need to prepare the variables and parameters to make the request data process.

In [128]:
# prepare for the monitoring logic
start_time = time() # note the system time when the program starts
#
# variables to handle the request
query = {'table':''}
url = ' https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M' # web to scrap
#
# keep the output clear
clear_output(wait = True)
#

With this code I going to get the data from the response.

In [129]:
# get the full data from the response
def get_data_response():
  data = response.text
  soup = BeautifulSoup(data, 'html.parser')
  table = soup.find_all('table')[0] # Grab the table
  new_table = pd.DataFrame(columns=['Postal Code','Borough','Neighbourhood'],index=range(0,181)) # I know the size and the columns names
  #

This code was prepared to extract the items required from the data.

In [130]:
def extratc_items():
  # get the items requested  
  row_marker = 0
  for row in table.find_all('tr'):
      column_marker = 0
      columns = row.find_all('td')
      for column in columns:
          new_table.iat[row_marker,column_marker] = column.get_text()[:-1]
          column_marker += 1
      row_marker += 1
  new_table=new_table.drop([0])
  #

In this code I did the data cleanning and fix the table according with the instructions.

In [131]:
def data_cleaning():
  # filtering the Borough from Toronto
  toronto_boroughs=['North York','East York','York','Scarborough','Etobicoke','Downtown Toronto','East Toronto','West Toronto','North Toronto','Central Toronto']
  new_table = new_table[new_table.Borough.isin(toronto_boroughs)]
  #
  # eliminating the rows with Borough == 'Not assigned'
  new_table.drop(new_table[new_table.Borough == 'Not assigned'].index, inplace=True)
  #
  # assigning the Borough like Neighbourhood for Neighbourhood == 'Not assigned'
  new_table['Neighbourhood'].mask(new_table['Neighbourhood'] == 'Not assigned', new_table['Borough'], inplace=True)
  #
  # reseting the indexes
  new_table = new_table.reset_index(drop=True)
  #

In order to prevent any isue with the response at the request, we need to check with a conditional 'if-else' to manage the errors or warnings.
Ones checked, we proceed to call our principal functions or present the isue info.

In [132]:
# make the request
response = requests.get(url, params=query)
#
# we got a valid response ?
if(response.ok):
  get_data_response
  extratc_items
  data_cleaning
else:
  # display a warning if there are any problems
  warn('Request #: {}, Failed with status code: {}'.format(request_count, response.status_code))
  #

If was a sussecful process, here we can see the final results with some info about it.

In [134]:
# output some logs for monitoring
elapsed_time = time() - start_time
clear_output()      
print('Scrap process complete\n')
print('Elapsed time: {} seconds, {} Valid Postal Codes Scraped.\n'.format(elapsed_time, len(new_table)))
print(new_table.describe(),'\n')
# output the results
new_table

Scrap process complete

Elapsed time: 162.38226127624512 seconds, 102 Valid Postal Codes Scraped.

       Postal Code     Borough Neighbourhood
count          102         102           102
unique         102           9            98
top            M9N  North York     Downsview
freq             1          24             4 



,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
97,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
98,M4Y,Downtown Toronto,Church and Wellesley
99,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
100,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."
